## Project: Analysing Currency Trends.

***

### 1. Data Collection

***

Importing libraries.

In [1]:
import yfinance as yf
import pandas as pd
import sqlite3


I'm getting data with a 60-minute interval covering 2 years, which is the maximum amount limited by Yahoo Finance. If I'm trying to get more data returned, "The requested range must be within the last 730 days".

***

#### Data related to the United States Dollar currency (USD represent by symbol **$**).

Raw data are taken from Yahoo Finance.

In [2]:
# Download 2 years of hourly data for USDGBP and USDEUR currency pairs
usd2y = yf.download(['USDGBP=X','USDEUR=X'], period='2y', interval='60m', group_by='ticker')
# Download 5 years of daily data for USDGBP and USDEUR currency pairs
usd5y = yf.download(['USDGBP=X','USDEUR=X'], period='5y', interval='1d', group_by='ticker')
# Download 5 days of 5 minute data for USDGBP and USDEUR currency pairs
usd5d = yf.download(['USDGBP=X','USDEUR=X'], period='5d', interval='5m', group_by='ticker')


[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed


Saving the data to CSV files.

In [3]:
# Save the data to CSV files
usdToSave = [usd2y, usd5y, usd5d]
filenames = ['usd2y.csv', 'usd5y.csv', 'usd5d.csv']
for i in range(3):
    usdToSave[i].to_csv(f'data/csv/{filenames[i]}')
print('Data saved to CSV files')


Data saved to CSV files


#### Data related to the Great British Pound (often referred to simply as the British Pounds) currency (GBP represent by symbol **£**).

Raw data are taken from Yahoo Finance.

In [4]:
# Download 2 years of hourly data for GBPUSD and EURUSD currency pairs
gbp2y = yf.download(['GBPUSD=X','EURUSD=X'], period='2y', interval='60m', group_by='ticker')
# Download 5 years of daily data for GBPUSD and EURUSD currency pairs
gbp5y = yf.download(['GBPUSD=X','EURUSD=X'], period='5y', interval='1d', group_by='ticker')
# Download 5 days of 5 minute data for GBPUSD and EURUSD currency pairs
gbp5d = yf.download(['GBPUSD=X','EURUSD=X'], period='5d', interval='5m', group_by='ticker')


[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed


Saving the data to CSV files.

In [5]:
# Save the data to CSV files
gbpToSave = [gbp2y, gbp5y, gbp5d]
filenames = ['gbp2y.csv', 'gbp5y.csv', 'gbp5d.csv']
for i in range(3):
    gbpToSave[i].to_csv(f'data/csv/{filenames[i]}')
print('Data saved to CSV files')

Data saved to CSV files


#### Data related to official currency of the Eurozone knows as Euro currency (EUR: represent by **€**). 

Raw data are taken from Yahoo Finance.

In [6]:
# Download 2 years of hourly data for EURUSD and EURGBP currency pairs
eur2y = yf.download(['EURUSD=X', 'EURGBP=X'], period='2y', interval='60m', group_by='ticker')
# Download 5 years of daily data for EURUSD and EURGBP currency pairs
eur5y = yf.download(['EURUSD=X', 'EURGBP=X'], period='5y', interval='1d', group_by='ticker')
# Download 5 days of 5 minute data for EURUSD and EURGBP currency pairs
eur5d = yf.download(['EURUSD=X', 'EURGBP=X'], period='5d', interval='5m', group_by='ticker')


[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed


Saving the data to CSV files.

In [7]:
# Save the data to CSV files
eurToSave = [eur2y, eur5y, eur5d]
filenames = ['eur2y.csv', 'eur5y.csv', 'eur5d.csv']
for i in range(3):
    eurToSave[i].to_csv(f'data/csv/{filenames[i]}')
print('Data saved to CSV files')


Data saved to CSV files


#### Data related to the Bitcoin cryptocurrency (BTC represented by symbol **₿**)

Raw data are taken from Yahoo Finance.

In [8]:
# Download 2 years of hourly data for BTCUSD, BTCGBP and BTCEUR currency pairs
btc2y = yf.download(['BTC-USD', 'BTC-GBP', 'BTC-EUR'], period='2y', interval='60m', group_by='ticker')
# Download 5 years of daily data for BTCUSD, BTCGBP and BTCEUR currency pairs
btc5y = yf.download(['BTC-USD', 'BTC-GBP', 'BTC-EUR'], period='5y', interval='1d', group_by='ticker')
# Download 5 days of 5 minute data for BTCUSD, BTCGBP and BTCEUR currency pairs
btc5d = yf.download(['BTC-USD', 'BTC-GBP', 'BTC-EUR'], period='5d', interval='5m', group_by='ticker')


[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  3 of 3 completed


Saving the data to CSV files.

In [9]:
# Save the data to CSV files
btcToSave = [btc2y, btc5y, btc5d]
filenames = ['btc2y.csv', 'btc5y.csv', 'btc5d.csv']
for i in range(3):
    btcToSave[i].to_csv(f'data/csv/{filenames[i]}')
print('Data saved to CSV files')


Data saved to CSV files


***

#### Preparing data to transfer to the Database.

Database Setup:

I will create an SQLite database named currency.db and set up individual tables for each currency pair and corresponding datasets. I will then insert the data from the CSV files into the tables. To ensure compatibility with the database structure, I need to flatten the multi-index columns in the data frames before inserting them into the database.

Reason for Downloading Separately:

When I attempted to download all the datasets at once, I noticed that some data was consistently missing or lost, even though the requested datasets were the same. This issue likely stemmed from limited access to Yahoo Finance's API, which can result in data loss when simultaneously querying large amounts of data.
By downloading the data separately, I can mitigate this problem and ensure I collect the better quality dataset for each currency pair without risking data corruption or loss due to API limitations.

Visual inspection of the data shows that the data is downloaded correctly. I noticed that the date format is different in 5 years of daily data than in 2 years of hourly data and 5 days of 5-minute data. I converted the date format to the same like in 2 years of hourly data and 5 days of 5-minute data.

Flattening the dataframes.

In [10]:
# List of DataFrames for 2 years, 5 years, and 5 days data
all2y = [usd2y, gbp2y, eur2y, btc2y]
all5y = [usd5y, gbp5y, eur5y, btc5y]
all5d = [usd5d, gbp5d, eur5d, btc5d]

# Reset index for all DataFrames
for df_list in [all2y, all5y, all5d]:
    for df in df_list:
        df.reset_index(inplace=True)  # Resets the index for each DataFrame

# Function to flatten multi-index columns into single-level columns
def flatten_columns(df):
    # Rename 'Date' to 'Datetime' for all DataFrames in all5y
    if 'Date' in df.columns:
        df.rename(columns={'Date': 'Datetime'}, inplace=True)
    
    # Flatten multi-index columns to single level
    df.columns = ['Datetime' if isinstance(col, tuple) and col[0] == 'Datetime' 
                  else '_'.join(col).strip() for col in df.columns]

# Apply flatten_columns function to each DataFrame in all lists
for df_list in [all2y, all5y, all5d]:
    for df in df_list:
        flatten_columns(df)
        
# Setting index for all DataFrames
for df_list in [all2y, all5y, all5d]:
    for df in df_list:
        df.set_index('Datetime', inplace=True)



Transfer CSVfiles to the SQLite Database.

In [11]:

# Connect to SQLite database (creates file if it doesn't exist)
conn = sqlite3.connect('data/db/currencies.db')

# List of DataFrames and their corresponding table names
dataframes = [(usd2y, 'usd2y'), (usd5y, 'usd5y'), (usd5d, 'usd5d'), 
              (gbp2y, 'gbp2y'), (gbp5y, 'gbp5y'), (gbp5d, 'gbp5d'), 
              (eur2y, 'eur2y'), (eur5y, 'eur5y'), (eur5d, 'eur5d'), 
              (btc2y, 'btc2y'), (btc5y, 'btc5y'), (btc5d, 'btc5d')]
# Loop to write each DataFrame to the SQLite database
for df, table_name in dataframes:
    df.to_sql(table_name, conn, if_exists='replace', index=True)
    
# Close the connection to SQLite
conn.close()

print("Data successfully written to SQLite database!")


Data successfully written to SQLite database!


Checking what tables contain the Database.

In [12]:
# Connect to the SQLite database
conn = sqlite3.connect('data/db/currencies.db')
cursor = conn.cursor()

# Query to list tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print the tables
print("Tables in the database:")
for table in tables:
    print(table[0])

# Close the connection
conn.close()

Tables in the database:
usd2y
usd5y
usd5d
gbp2y
gbp5y
gbp5d
eur2y
eur5y
eur5d
btc2y
btc5y
btc5d


Testing  query to the Database.

In [13]:
# Connect to the SQLite database
conn = sqlite3.connect('data/db/currencies.db')

# Read data from the table into a pandas DataFrame
query = "SELECT * FROM gbp5y LIMIT 5;"  # Adjust for your table name
df = pd.read_sql(query, conn)
df.set_index('Datetime', inplace=True)

# Display the DataFrame

df

,GBPUSD=X_Open,GBPUSD=X_High,GBPUSD=X_Low,GBPUSD=X_Close,GBPUSD=X_Adj Close,GBPUSD=X_Volume,EURUSD=X_Open,EURUSD=X_High,EURUSD=X_Low,EURUSD=X_Close,EURUSD=X_Adj Close,EURUSD=X_Volume
Datetime,,,,,,,,,,,,
2019-12-13 00:00:00,1.347654,1.350986,1.330955,1.347527,1.347527,0.0,1.118343,1.118693,1.111889,1.118606,1.118606,0
2019-12-16 00:00:00,1.335524,1.341706,1.332623,1.335327,1.335327,0.0,1.112520,1.115785,1.112347,1.112446,1.112446,0
2019-12-17 00:00:00,1.327175,1.330495,1.310410,1.327246,1.327246,0.0,1.113710,1.117319,1.113090,1.113697,1.113697,0
2019-12-18 00:00:00,1.312215,1.312560,1.306233,1.312215,1.312215,0.0,1.115150,1.115150,1.111136,1.115138,1.115138,0
2019-12-19 00:00:00,1.308472,1.312956,1.299663,1.308729,1.308729,0.0,1.111593,1.114243,1.110988,1.111593,1.111593,0


Understanding collected data is kind of important. We can identify columns called:
* Open: Price at the start of the trading period.
* High: Highest price during the trading period.
* Low: Lowest price during the trading period.
* Close: Price at the end of the trading period.
* Adj Close: Close price adjusted for events like splits or dividends (not relevant in this case).
* Volume: Total amount of the asset traded during the period.

Display the first 3 row are commented out and was use only to check Data Frame and discover column names.

***

### 2. Data Cleaning

I chose to analyse OHLC data (Open, High, Low, Close) because it provides a comprehensive view of price movements, which is essential for identifying market trends and making informed trading decisions.

I will omit other data, such as volume or adjusted close, as they are less relevant for understanding price action and trend analysis. After examining the data, it is clear that the Adj Close column contains the same values as the Close column, and the Volume column is empty.

Formatting Data Frames for easier manipulation 

In [14]:
# Columns to drop (targeting columns that have '_Adj Close' or '_Volume' suffixes)
columnsToDrop = ['_Adj Close', '_Volume']

# List of DataFrames for each asset
dfColumnsDroppedList = [usd2y, gbp2y, eur2y, btc2y, usd5y, gbp5y, eur5y, btc5y, usd5d, gbp5d, eur5d, btc5d]

# Iterate over each DataFrame and drop columns that contain '_Adj Close' or '_Volume'
for df in dfColumnsDroppedList:
    df.drop([col for col in df.columns if any(drop in col for drop in columnsToDrop)], axis=1, inplace=True)

# Verify the result (Optional)
usd2y.head(3)


,USDGBP=X_Open,USDGBP=X_High,USDGBP=X_Low,USDGBP=X_Close,USDEUR=X_Open,USDEUR=X_High,USDEUR=X_Low,USDEUR=X_Close
Datetime,,,,,,,,
2022-12-14 00:00:00,0.80948,0.81027,0.80921,0.80951,0.9410,0.9410,0.9398,0.9399
2022-12-14 01:00:00,0.80937,0.80991,0.80932,0.80953,0.9399,0.9403,0.9396,0.9401
2022-12-14 02:00:00,0.80959,0.81019,0.80930,0.80991,0.9401,0.9408,0.9398,0.9407


In [ ]:
# Merge the DataFrames using 'Datetime' column and add suffixes
mergedFiatCurrencies2y = usd2y.merge(eur2y, how='outer', on='Datetime', suffixes=('_USD', '_EUR')).merge(gbp2y, how='outer', on='Datetime', suffixes=('_EUR', '_GBP'))
mergedFiatCurrencies5d = usd5d.merge(eur5d, how='outer', on='Datetime', suffixes=('_USD', '_EUR')).merge(gbp5d, how='outer', on='Datetime', suffixes=('_EUR', '_GBP'))
mergedFiatCurrencies5y = usd5y.merge(eur5y, how='outer', on='Datetime', suffixes=('_USD', '_EUR')).merge(gbp5y, how='outer', on='Datetime', suffixes=('_EUR', '_GBP'))


,USDEUR=X_Open,USDEUR=X_High,USDEUR=X_Low,USDEUR=X_Close,USDGBP=X_Open,USDGBP=X_High,USDGBP=X_Low,USDGBP=X_Close,EURUSD=X_Open_EUR,EURUSD=X_High_EUR,...,EURGBP=X_Low,EURGBP=X_Close,GBPUSD=X_Open,GBPUSD=X_High,GBPUSD=X_Low,GBPUSD=X_Close,EURUSD=X_Open_GBP,EURUSD=X_High_GBP,EURUSD=X_Low_GBP,EURUSD=X_Close_GBP
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-12-13,0.89418,0.89937,0.89390,0.89397,0.74203,0.75134,0.74020,0.74210,1.118343,1.118693,...,0.8276,0.83004,1.347654,1.350986,1.330955,1.347527,1.118343,1.118693,1.111889,1.118606
2019-12-16,0.89886,0.89900,0.89623,0.89892,0.74877,0.75040,0.74532,0.74888,1.112520,1.115785,...,0.8304,0.83303,1.335524,1.341706,1.332623,1.335327,1.112520,1.115785,1.112347,1.112446
2019-12-17,0.89790,0.89840,0.89500,0.89791,0.75348,0.76312,0.75160,0.75344,1.113710,1.117319,...,0.8380,0.83905,1.327175,1.330495,1.310410,1.327246,1.113710,1.117319,1.113090,1.113697


#### Fill missing values using forward fill (ffill) method

In [16]:
# List of DataFrames
dataframes = [mergedFiatCurrencies2y, mergedFiatCurrencies5y, mergedFiatCurrencies5d]

# Forward fill missing values for each DataFrame in the list
for df in dataframes:
    df.ffill(inplace=True)


#### Checking Datetime format if it is in expected form: '%Y-%m-%d %H:%M:%S'.

In [17]:
# Check if 'Datetime' is in any level of the MultiIndex
if 'Datetime' in mergedFiatCurrencies2y.index.names:
    # Extract the 'Datetime' level from the MultiIndex
    datetime_column = mergedFiatCurrencies2y.index.get_level_values('Datetime')
    
    # Check if the 'Datetime' index is in datetime64 format
    if pd.api.types.is_datetime64_any_dtype(datetime_column):
        # Display the format of the 'Datetime' index
        datetime_format = datetime_column[0].strftime('%Y-%m-%d %H:%M:%S')  # You can adjust the format as needed
        print(f"The 'Datetime' index is correctly formatted as datetime64.\nExample of the format: {datetime_format}\nThere is no action required.")
    else:
        print("The 'Datetime' index is not in datetime64 format.\nPlease convert it.")


The 'Datetime' index is correctly formatted as datetime64.
Example of the format: 2022-12-14 00:00:00
There is no action required.


#### Normalize Bitcoin exchange rates

In [18]:
# Normalizing the data

# List of DataFrames to normalize
btcDataFrames = [btc2y, btc5y, btc5d]

# Normalize each DataFrame and store them in a list
normalisedBtcFrames = [df.div(df.iloc[0]).mul(100) for df in btcDataFrames]

#Fillin missing values
normalisedBtcFrames = [df.ffill() for df in normalisedBtcFrames]

# Assign normalised DataFrames to distinct variables
normalisedbtc2y, normalisedbtc5y, normalisedbtc5d = normalisedBtcFrames[0], normalisedBtcFrames[1], normalisedBtcFrames[2]
